## Initialize

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

from openai import OpenAI

model="claude-opus-4-20250514"

client_config = {
    "type": "ttt",
    "client_type": "anthropic",
    "model": model
    }


### Generate

In [4]:
import anthropic

model = "claude-opus-4-20250514"

## Using anthropic client ────────────────────────────────────────────────────────────────────────

print("\n\nNormal:\n")

client = anthropic.Anthropic()
response = client.messages.create(
    model=model,
    max_tokens=1000,
    messages=[
        {
            "role": "user",
            "content": "tell me a one sentence story",
        },
    ],
)
print(f"Raw:\n{response.model_dump()}\nExtracted:\n{response.content[0].text}")

## Using OpenAI-Compatible client ───────────────────────────────────────────────────────────────────

print("\n\nPatched:\n")

from gai.llm.openai import OpenAI
client = OpenAI(client_config=client_config)
response = client.chat.completions.create(
    model=client_config["model"], 
    messages=[{"role":"user","content":"tell me a one sentence story"}]
)
print(f"Raw:\n{response.model_dump()}\nExtracted:\n{response.extract()}")

## Using OpenAI-Compatible async client ───────────────────────────────────────────────────────────────────

print("\n\nAsync:\n")

from gai.llm.openai import AsyncOpenAI
client = AsyncOpenAI(client_config=client_config)
response = await client.chat.completions.create(
    model=client_config["model"], 
    messages=[{"role":"user","content":"tell me a one sentence story"}]
)
print(f"Raw:\n{response.model_dump()}\nExtracted:\n{response.extract()}")



Normal:

Raw:
{'id': 'msg_01Fok6pmtuoFEDnGKhkXcy9x', 'content': [{'citations': None, 'text': "The old lighthouse keeper discovered the light he'd been tending for forty years had never worked, but the ships had always found their way home anyway.", 'type': 'text'}], 'model': 'claude-opus-4-20250514', 'role': 'assistant', 'stop_reason': 'end_turn', 'stop_sequence': None, 'type': 'message', 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 13, 'output_tokens': 33, 'server_tool_use': None, 'service_tier': 'standard'}}
Extracted:
The old lighthouse keeper discovered the light he'd been tending for forty years had never worked, but the ships had always found their way home anyway.


Patched:

Raw:
{'id': 'chatcmpl-9b0b0a5b-860b-4a06-bd87-9f435b94eee9', 'choices': [{'finish_reason': 'stop', 'index': 0, 'logprobs': None, 'message': {'content': 'As the last library on Earth crumbled to dust, an old woman tucked the final book under her arm and began te

---

### Streaming

In [4]:
import anthropic

model = "claude-opus-4-20250514"

## Using anthropic client ────────────────────────────────────────────────────────────────────────

print("\n\nNormal:\n")

import json
client = anthropic.Anthropic()
response = client.messages.create(
    model=model,
    max_tokens=1000,
    messages=[
        {
            "role": "user",
            "content": "Tell me a one sentence story",
        },
    ],
    stream=True
)
chunks = []
for chunk in response:
    if chunk.type=="content_block_delta":
        chunks.append(chunk.delta.text)
print(json.dumps(chunks,indent=4))
            
## Using gai.llm.openai client ───────────────────────────────────────────────────────────────────

print("\n\nPatched:\n")

from gai.llm.openai import OpenAI
client = OpenAI(client_config=client_config)
response = client.chat.completions.create(
    model=client_config["model"], 
    messages=[{"role":"user","content":"tell me a one sentence story"}],
    stream=True
)
chunks = []
for chunk in response:
    chunk = chunk.extract()
    if (isinstance(chunk, str)):
        print(chunk, end="", flush=True)


import os
from dotenv import load_dotenv
load_dotenv()

from openai import OpenAI

model="claude-opus-4-20250514"

client_config = {
    "type": "ttt",
    "client_type": "anthropic",
    "model": model
    }

# # Using gai.llm.openai async client ───────────────────────────────────────────────────────────────────

print("\n\nAsync:\n")

from gai.llm.openai import AsyncOpenAI
client = AsyncOpenAI(client_config=client_config)
response = await client.chat.completions.create(
    model=client_config["model"], 
    messages=[{"role":"user","content":"tell me a one sentence story"}],
    stream=True
)
chunks = []
async for chunk in response:
    if chunk:
        chunk = chunk.extract()
        if (isinstance(chunk, str)):
            print(chunk, end="", flush=True)
    




Normal:

[
    "The old lighthouse",
    " keeper discovered that the strange signals he'd been receiving for",
    " forty years were actually replies to messages he'd forgotten",
    " sending into space as a lonely child."
]


Patched:

The old lighthouse keeper discovered that the mysterious signals he'd been receiving for forty years were actually from his own future self, warning him not to retire next Tuesday.

Async:

The old lighthouse keeper discovered that the mysterious lights appearing on the horizon each night weren't ships in distress, but his daughter sailing home after twenty years to finally forgive him.

---
### Tool Call

In [2]:
import anthropic

model = "claude-opus-4-20250514"

## Using anthropic client ────────────────────────────────────────────────────────────────────────

print("\n\nNormal:\n")

client = anthropic.Anthropic()
response = client.messages.create(
    model=model, 
    messages=[{"role": "user", "content": "What is the current time in Singapore?"}],
    tools=[
        {
            "name": "google",
            "description": "The 'google' function is a powerful tool that allows the AI to gather external information from the internet using Google search. It can be invoked when the AI needs to answer a question or provide information that requires up-to-date, comprehensive, and diverse sources which are not inherently known by the AI. For instance, it can be used to find current date, current news, weather updates, latest sports scores, trending topics, specific facts, or even the current date and time. The usage of this tool should be considered when the user's query implies or explicitly requests recent or wide-ranging data, or when the AI's inherent knowledge base may not have the required or most current information. The 'search_query' parameter should be a concise and accurate representation of the information needed.",
            "input_schema": {
                "type": "object",
                "properties": {
                    "search_query": {
                        "type": "string",
                        "description": "The search query to search google with. For example, to find the current date or time, use 'current date' or 'current time' respectively."
                    }
                },
                "required": ["search_query"]
            }
        }
    ],
    max_tokens=1000,
)
import json
print(json.dumps(response.model_dump(), indent=4))

## Using OpenAI-Compatible client ───────────────────────────────────────────────────────────────────

print("\n\nPatched:\n")

from gai.llm.openai import OpenAI
client = OpenAI(client_config=client_config)
response = client.chat.completions.create(
    model=client_config["model"],
    messages=[{"role": "user", "content": "What is the current time in Singapore?"}],
    tools=[
        {
            "type": "function",
            "function": {
                "name": "google",
                "description": "The 'google' function is a powerful tool that allows the AI to gather external information from the internet using Google search. It can be invoked when the AI needs to answer a question or provide information that requires up-to-date, comprehensive, and diverse sources which are not inherently known by the AI. For instance, it can be used to find current date, current news, weather updates, latest sports scores, trending topics, specific facts, or even the current date and time. The usage of this tool should be considered when the user's query implies or explicitly requests recent or wide-ranging data, or when the AI's inherent knowledge base may not have the required or most current information. The 'search_query' parameter should be a concise and accurate representation of the information needed.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "search_query": {
                            "type": "string",
                            "description": "The search query to search google with. For example, to find the current date or time, use 'current date' or 'current time' respectively."
                        }
                    },
                    "required": ["search_query"]
                }
            }
        }
    ]
)

# For anthropic, the result will be returned as serialized array of ContentBlocks as ChatCompletionMessage's content.

print(response.extract())

## Using OpenAI-Compatible async client ───────────────────────────────────────────────────────────────────

print("\n\nAsync:\n")

from gai.llm.openai import AsyncOpenAI
client = AsyncOpenAI(client_config=client_config)
response = await client.chat.completions.create(
    model=client_config["model"],
    messages=[{"role": "user", "content": "What is the current time in Singapore?"}],
    tools=[
        {
            "type": "function",
            "function": {
                "name": "google",
                "description": "The 'google' function is a powerful tool that allows the AI to gather external information from the internet using Google search. It can be invoked when the AI needs to answer a question or provide information that requires up-to-date, comprehensive, and diverse sources which are not inherently known by the AI. For instance, it can be used to find current date, current news, weather updates, latest sports scores, trending topics, specific facts, or even the current date and time. The usage of this tool should be considered when the user's query implies or explicitly requests recent or wide-ranging data, or when the AI's inherent knowledge base may not have the required or most current information. The 'search_query' parameter should be a concise and accurate representation of the information needed.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "search_query": {
                            "type": "string",
                            "description": "The search query to search google with. For example, to find the current date or time, use 'current date' or 'current time' respectively."
                        }
                    },
                    "required": ["search_query"]
                }
            }
        }
    ],
    tool_choice="required"
)
print(response.extract())



Normal:

{
    "id": "msg_016918vfNbJ4rhurm8ish4vP",
    "content": [
        {
            "citations": null,
            "text": "I'll help you find the current time in Singapore.",
            "type": "text"
        },
        {
            "id": "toolu_01V6XGw6WjgWwcbuTggpmfRV",
            "input": {
                "search_query": "current time in Singapore"
            },
            "name": "google",
            "type": "tool_use"
        }
    ],
    "model": "claude-opus-4-20250514",
    "role": "assistant",
    "stop_reason": "tool_use",
    "stop_sequence": null,
    "type": "message",
    "usage": {
        "cache_creation_input_tokens": 0,
        "cache_read_input_tokens": 0,
        "input_tokens": 578,
        "output_tokens": 68,
        "server_tool_use": null,
        "service_tier": "standard"
    }
}


Patched:

[{'citations': None, 'text': "I'll help you find the current time in Singapore.", 'type': 'text'}, {'id': 'toolu_016E6Hp1oykLtiSvBC6AAsuY', 'input': {'s

#### Parameterless Tool Call

In [3]:
import anthropic

model = "claude-opus-4-20250514"

from gai.llm.openai import AsyncOpenAI
client = AsyncOpenAI(client_config={
    "client_type": "anthropic",
    "model": model,
})
response = await client.chat.completions.create(
    model=client_config["model"],
    messages=[{"role": "user", "content": "What is the current time in Singapore?"}],
    tools=[
        {
            "type": "function",
            "function": {
                "name": "get_current_time",
                "description": "Get current time in Singapore",
                "parameters": {
                    "type": "object",
                    "properties": {},
                    "required": []
                }
            }
        }
    ],
    tool_choice="required"
)
print(response.extract())

[{"citations": null, "text": "I'll get the current time in Singapore for you.", "type": "text"}, {"id": "toolu_0168bhSBi2TpewxdCn2oD657", "input": {}, "name": "get_current_time", "type": "tool_use"}]


#### Streaming Tool Call

Can't think of a good reason why tool call should be streamed but here it is provided to maintain compatibility with the original API.



In [6]:
model = "claude-opus-4-20250514"

# ## Using OpenAI-Compatible client ───────────────────────────────────────────────────────────────────
from rich import print as rprint
from gai.llm.openai import AsyncOpenAI
client = AsyncOpenAI(client_config=client_config)
response = await client.chat.completions.create(
    model=client_config["model"],
    messages=[{"role": "user", "content": "What is the current time in Singapore?"}],
    tools=[
        {
            "type": "function",
            "function": {
                "name": "google",
                "description": "The 'google' function is a powerful tool that allows the AI to gather external information from the internet using Google search. It can be invoked when the AI needs to answer a question or provide information that requires up-to-date, comprehensive, and diverse sources which are not inherently known by the AI. For instance, it can be used to find current date, current news, weather updates, latest sports scores, trending topics, specific facts, or even the current date and time. The usage of this tool should be considered when the user's query implies or explicitly requests recent or wide-ranging data, or when the AI's inherent knowledge base may not have the required or most current information. The 'search_query' parameter should be a concise and accurate representation of the information needed.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "search_query": {
                            "type": "string",
                            "description": "The search query to search google with. For example, to find the current date or time, use 'current date' or 'current time' respectively."
                        }
                    },
                    "required": ["search_query"]
                }
            }
        }
    ],
    stream=True,
    tool_choice="required"
)

# The stream starts by returning a series of text chunks forming the text_block and ends with a final Message which returns the entirety of the result
# corresponding to the result returned from an unstreamed call.
content=None
async for chunk in response:
    if chunk:
        print(chunk)
        print()

# async for chunk in response:
#     if chunk:
#         chunk = chunk.extract()
#     if isinstance(chunk,str):
#         print(chunk,end="",flush=True)
#     else:
#         rprint(f"[green]content={chunk}[/]")
        


ChatCompletionChunk(id='chatcmpl-80d3bdbf-c73c-4cb4-884c-20fa2541bf44', choices=[Choice(delta=ChoiceDelta(content='', function_call=None, refusal=None, role='assistant', tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1749561267, model='anthropic', object='chat.completion.chunk', service_tier=None, system_fingerprint=None, usage=None, extract=<function attach_extractor_async.<locals>.extract_chunk_logic.<locals>.<lambda> at 0x756373569ab0>)

ChatCompletionChunk(id='chatcmpl-401458cc-288f-445e-aa26-5a7aa578a75c', choices=[Choice(delta=ChoiceDelta(content="I'll help you find the current time", function_call=None, refusal=None, role=None, tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1749561267, model='anthropic', object='chat.completion.chunk', service_tier=None, system_fingerprint=None, usage=None, extract=<function attach_extractor_async.<locals>.extract_chunk_logic.<locals>.<lambda> at 0x756371aef760>)

ChatCompletionChunk(id='chatcmpl

When tool call is not needed.

In [9]:
model = "claude-opus-4-20250514"

# ## Using OpenAI-Compatible client ───────────────────────────────────────────────────────────────────

print("\n\nPatched:\n")

from gai.llm.openai import AsyncOpenAI
client = AsyncOpenAI(client_config=client_config)
response = await client.chat.completions.create(
    model=client_config["model"],
    messages=[{"role": "user", "content": "Tell me a one sentence story"}],
    tools=[
        {
            "type": "function",
            "function": {
                "name": "google",
                "description": "The 'google' function is a powerful tool that allows the AI to gather external information from the internet using Google search. It can be invoked when the AI needs to answer a question or provide information that requires up-to-date, comprehensive, and diverse sources which are not inherently known by the AI. For instance, it can be used to find current date, current news, weather updates, latest sports scores, trending topics, specific facts, or even the current date and time. The usage of this tool should be considered when the user's query implies or explicitly requests recent or wide-ranging data, or when the AI's inherent knowledge base may not have the required or most current information. The 'search_query' parameter should be a concise and accurate representation of the information needed.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "search_query": {
                            "type": "string",
                            "description": "The search query to search google with. For example, to find the current date or time, use 'current date' or 'current time' respectively."
                        }
                    },
                    "required": ["search_query"]
                }
            }
        }
    ],
    stream=True,
)

async for chunk in response:
    if chunk:
        print(chunk)
        print()
    




Patched:

ChatCompletionChunk(id='chatcmpl-d34aeb69-6382-4ecf-a749-f71df87bb8ee', choices=[Choice(delta=ChoiceDelta(content='', function_call=None, refusal=None, role='assistant', tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1749561414, model='anthropic', object='chat.completion.chunk', service_tier=None, system_fingerprint=None, usage=None, extract=<function attach_extractor_async.<locals>.extract_chunk_logic.<locals>.<lambda> at 0x756370fd15a0>)

ChatCompletionChunk(id='chatcmpl-71be066d-6d2a-4294-8295-5265fa4b3554', choices=[Choice(delta=ChoiceDelta(content='I', function_call=None, refusal=None, role=None, tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1749561414, model='anthropic', object='chat.completion.chunk', service_tier=None, system_fingerprint=None, usage=None, extract=<function attach_extractor_async.<locals>.extract_chunk_logic.<locals>.<lambda> at 0x756370fd1900>)

ChatCompletionChunk(id='chatcmpl-e3a5cd65-7675-416b-ae

---

### Structured Output


In [4]:

from pydantic import BaseModel
class Book(BaseModel):
    title: str
    summary: str
    author: str
    published_year: int
data = """Foundation is a science fiction novel by American writer
        Isaac Asimov. It is the first published in his Foundation Trilogy (later
        expanded into the Foundation series). Foundation is a cycle of five
        interrelated short stories, first published as a single book by Gnome Press
        in 1951. Collectively they tell the early story of the Foundation,
        an institute founded by psychohistorian Hari Seldon to preserve the best
        of galactic civilization after the collapse of the Galactic Empire."""       

import anthropic

model = "claude-opus-4-20250514"

## Using anthropic client ────────────────────────────────────────────────────────────────────────

print("\n\nNormal:\n")

client = anthropic.Anthropic()

# Use tool calls to return structured output
book_tool = {
    "name": "structured_output",
    "description": "Extract and structure book information",
    "input_schema": Book.model_json_schema()
}

response = client.messages.create(
    model=model,
    max_tokens=1000,
    temperature=0,
    tools=[book_tool],
    messages=[
        {
            "role": "user",
            "content": f"Extract structured output from this: {data}"
        }
    ]
)
print(response.model_dump_json(indent=4))

## Using OpenAI-Compatible client ───────────────────────────────────────────────────────────────────

print("\n\nPatched:\n")

from gai.llm.openai import OpenAI
client = OpenAI(client_config=client_config)
response = client.beta.chat.completions.parse(
    model=client_config["model"], 
    response_format=Book,
    messages=[{"role":"user","content":data}]
    )
print(response.model_dump_json(exclude={"extract"},indent=4))

print("\n\nExtracted:\n")
print(response.extract())

## Using OpenAI-Compatible async client ───────────────────────────────────────────────────────────────────

print("\n\nPatched:\n")

from gai.llm.openai import AsyncOpenAI
client = AsyncOpenAI(client_config=client_config)
response = await client.beta.chat.completions.parse(
    model=client_config["model"], 
    response_format=Book,
    messages=[{"role":"user","content":data}]
    )
print(response.model_dump_json(exclude={"extract"},indent=4))

print("\n\nExtracted:\n")
print(response.extract())




Normal:

{
    "id": "msg_015oMv9grVTpyJy8Pqu87GEB",
    "content": [
        {
            "citations": null,
            "text": "I'll extract the structured information from this text about the book \"Foundation\".",
            "type": "text"
        },
        {
            "id": "toolu_015qCs4okfv3CdrNQ6hBSQ8Y",
            "input": {
                "title": "Foundation",
                "author": "Isaac Asimov",
                "published_year": 1951,
                "summary": "Foundation is a science fiction novel that is the first published in the Foundation Trilogy (later expanded into the Foundation series). It consists of a cycle of five interrelated short stories that collectively tell the early story of the Foundation, an institute founded by psychohistorian Hari Seldon to preserve the best of galactic civilization after the collapse of the Galactic Empire."
            },
            "name": "structured_output",
            "type": "tool_use"
        }
    ],
    "mo

/workspaces/gai-sdk/.venv/lib/python3.10/site-packages/pydantic/main.py:519: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `str` - serialized value may not be as expected [input_value=[{'citations': None, 'tex...t', 'type': 'tool_use'}], input_type=list])
  return self.__pydantic_serializer__.to_json(


{
    "id": "chatcmpl-bf835a5c-af85-465e-9d71-637b9a2afb4a",
    "choices": [
        {
            "finish_reason": "stop",
            "index": 0,
            "logprobs": null,
            "message": {
                "content": [
                    {
                        "citations": null,
                        "text": "I'll extract the information about the Foundation novel and return it in JSON format.",
                        "type": "text"
                    },
                    {
                        "id": "toolu_01X8Zn8J3wjkdEPzci7xpNW6",
                        "input": {
                            "title": "Foundation",
                            "summary": "Foundation is a science fiction novel that is the first published in the Foundation Trilogy (later expanded into the Foundation series). It is a cycle of five interrelated short stories, first published as a single book by Gnome Press in 1951. The stories collectively tell the early story of the Foundation

---

### Serialization

In [9]:
from typing import List, Union, Annotated
from anthropic.types import TextBlock, ToolUseBlock
from pydantic import Field,BaseModel,TypeAdapter
MessageBlock = Annotated[Union[TextBlock, ToolUseBlock],Field(discriminator="type")]

## Using Opus ───────────────────────────────────────────────────────────────────

print("\n\nOpus-4-20250514:\n")


MessageList = TypeAdapter(List[MessageBlock])
    
messages = [
    TextBlock(type="text",text="Hello, how are you?"),
    ToolUseBlock(
        id="aaa",
        type="tool_use",
        name="google",
        input={"search_query": "current weather in Singapore"}
    )]

message_list = MessageList.dump_json(messages)
print(message_list)

loaded = MessageList.validate_json(message_list)
print(loaded)
    



Opus-4-20250514:

b'[{"citations":null,"text":"Hello, how are you?","type":"text"},{"id":"aaa","input":{"search_query":"current weather in Singapore"},"name":"google","type":"tool_use"}]'
[TextBlock(citations=None, text='Hello, how are you?', type='text'), ToolUseBlock(id='aaa', input={'search_query': 'current weather in Singapore'}, name='google', type='tool_use')]
